<a href="https://colab.research.google.com/github/gizemkaratas/Termin-Tarihi-Tahmin/blob/main/termin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [157]:
!pip install lifetimes
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
from sklearn.preprocessing import MinMaxScaler

In [158]:
df=pd.read_excel("aster.xlsx")

In [159]:
df.head(5)

,PARTIYIL,Parti Açılış Tarihi,BOYASIPARISYIL,Sipariş Grup Adı,Kilo,Net Sevk Kilo,Planlanan Kilo,Metre,Net Sevk Metre,Reçete Adı,İşlem Grubu,İşlem Tipi,Planlanan İmalat Tarihi,Termin Tarihi,Termin Gecikme Gün,İmalat Gecikme Gün,Onay Gün,Reçete Gün,işleme alınma-onay gün,işlem süresi,sevk-işlem,brm boya (kg/m),metre başına düşen boya maliyeti,metre başına düşen boya süresi,Fire Oranı,Reçete Tarihi,Ham Stok Giriş Tarihi,Onay Tarihi,MAKINAID,Fire Oranı.1,Lab Reçete Boya Maliyeti,Boya Maliyeti (Kg),Kimyasal Maliyeti (Kg),En İstenen,Gramaj İstenen,Kazana İlk Giriş Tarihi,Son Çıkış Tarihi,Siparis İşlem Grubu,Sevk İrsaliye Tarihleri
0,2023,2023-01-16,2023,LCW,20.0000,15.2000,20.0000,0,58.9150,R2E 18-3919 TCX.QTX,Renkli,Normal,2023-01-16,2023-01-16,8.0000,4.3603,29,3.7048,4.3603,3.7386,0.0989,0.3395,0.1027,0.0635,0.2400,2023-01-19 16:54:58,2022-12-18,2023-01-16,0050 HT/3,24.0000,9.9035,6.0489,2.4044,185,145,2023-01-20 08:38:53,2023-01-24 02:22:25,Renkli,24.01.2023
1,2023,2023-01-16,2023,LCW,17.0000,14.3000,17.0000,0,44.6880,R2E 18-3919 TCX.QTX,Renkli,Normal,2023-01-16,2023-01-16,8.0000,4.3604,66,3.7048,4.3604,3.7386,0.0990,0.3804,0.1354,0.0837,0.1588,2023-01-19 16:54:58,2022-11-11,2023-01-16,0050 HT/3,15.8800,9.9035,6.0489,2.4044,185,190,2023-01-20 08:38:56,2023-01-24 02:22:35,Renkli,24.01.2023
2,2023,2023-01-16,2023,LCW,10.0000,9.0000,10.0000,0,32.1430,R2E 18-3919 TCX.QTX,Renkli,Normal,2023-01-16,2023-01-16,8.0000,4.3604,102,3.7048,4.3604,3.7387,0.0991,0.3111,0.1882,0.1163,0.1000,2023-01-19 16:54:58,2022-10-06,2023-01-16,0050 HT/3,10.0000,9.9035,6.0489,2.4044,110,260,2023-01-20 08:38:59,2023-01-24 02:22:42,Renkli,24.01.2023
3,2023,2023-01-18,2023,VERY MAIN,306.8000,273.5000,306.8000,0,1005.5150,NAVY BLAZER,Renkli,Normal,2023-01-18,2023-01-11,14.0000,3.8736,-1,3.5086,3.8736,3.1628,0.0364,0.3051,0.0044,0.0031,0.1085,2023-01-21 12:12:24,2023-01-19,2023-01-18,0450 HT/1,10.8500,7.7612,4.4498,3.3353,180,160,2023-01-21 20:58:00,2023-01-25 00:52:26,Renkli,25.01.2023
4,2023,2023-01-18,2023,VERY MAIN,28.0000,25.0000,28.0000,0,68.4930,NAVY BLAZER,Renkli,Normal,2023-01-18,2023-01-11,14.0000,3.8736,1,3.5086,3.8736,3.1629,0.0365,0.4088,0.0650,0.0462,0.1071,2023-01-21 12:12:24,2023-01-17,2023-01-18,0450 HT/1,10.7100,7.7612,4.4498,3.3353,160,220,2023-01-21 20:58:00,2023-01-25 00:52:33,Renkli,25.01.2023


In [160]:
df.isnull().sum()


,0
PARTIYIL,0
Parti Açılış Tarihi,0
BOYASIPARISYIL,0
Sipariş Grup Adı,1284
Kilo,0
Net Sevk Kilo,0
Planlanan Kilo,0
Metre,0
Net Sevk Metre,0
Reçete Adı,3


In [161]:
df.shape

(5076, 39)

In [162]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
PARTIYIL,5076.0000,2023.0000,2023.0000,2023.0000,2023.0000,2023.0000,2023.0000,0.0000
Parti Açılış Tarihi,5076,2023-06-30 10:30:04.255319296,2023-01-02 00:00:00,2023-03-28 00:00:00,2023-07-10 00:00:00,2023-10-04 00:00:00,2023-12-29 00:00:00,NaN
BOYASIPARISYIL,5076.0000,2022.9413,2022.0000,2023.0000,2023.0000,2023.0000,2023.0000,0.2351
Kilo,5076.0000,327.0285,1.0000,38.0000,145.0000,556.6250,2039.5000,377.0254
Net Sevk Kilo,5076.0000,294.5994,0.0100,30.5000,128.5500,504.2750,1930.5000,345.2230
Planlanan Kilo,5076.0000,299.9964,0.0000,20.0000,107.7500,517.0000,2058.0000,385.6563
Metre,5076.0000,0.4728,0.0000,0.0000,0.0000,0.0000,2400.0000,33.6861
Net Sevk Metre,5076.0000,883.7240,0.0170,85.7665,335.1695,1155.1580,677000.0000,9541.0571
Planlanan İmalat Tarihi,5076,2023-06-30 10:30:04.255319296,2023-01-02 00:00:00,2023-03-28 00:00:00,2023-07-10 00:00:00,2023-10-04 00:00:00,2023-12-29 00:00:00,NaN
Termin Tarihi,5076,2023-06-12 13:11:46.382978560,2022-01-30 00:00:00,2023-03-15 00:00:00,2023-06-08 00:00:00,2023-09-23 00:00:00,2024-10-07 00:00:00,NaN


In [163]:
# prompt: boş değerleri sil

df.dropna(inplace=True)


In [164]:
# prompt: "Fire Oranı" ve "Fire Oranı.1" de negatif değerleri pozitif hale çevirif dfye kaydet

# "Fire Oranı" ve "Fire Oranı.1" sütunlarındaki negatif değerleri pozitif yapma
df['Fire Oranı'] = df['Fire Oranı'].abs()
df['Fire Oranı.1'] = df['Fire Oranı.1'].abs()


In [165]:
# prompt: Parti Açılış Tarihi,Planlanan İmalat Tarihi,Termin Tarihi,Reçete Tarihi,Ham Stok Giriş Tarihi,Onay Tarihi,Kazana İlk Giriş Tarihi,Son Çıkış Tarihi  datetime a çevirelim

import pandas as pd
# Tarih sütunlarını datetime'a çevirme
df['Parti Açılış Tarihi'] = pd.to_datetime(df['Parti Açılış Tarihi'], errors='coerce')
df['Planlanan İmalat Tarihi'] = pd.to_datetime(df['Planlanan İmalat Tarihi'], errors='coerce')
df['Termin Tarihi'] = pd.to_datetime(df['Termin Tarihi'], errors='coerce')
df['Reçete Tarihi'] = pd.to_datetime(df['Reçete Tarihi'], errors='coerce')
df['Ham Stok Giriş Tarihi'] = pd.to_datetime(df['Ham Stok Giriş Tarihi'], errors='coerce')
df['Onay Tarihi'] = pd.to_datetime(df['Onay Tarihi'], errors='coerce')
df['Kazana İlk Giriş Tarihi'] = pd.to_datetime(df['Kazana İlk Giriş Tarihi'], errors='coerce')
df['Son Çıkış Tarihi'] = pd.to_datetime(df['Son Çıkış Tarihi'], errors='coerce')


In [166]:
df.head()

,PARTIYIL,Parti Açılış Tarihi,BOYASIPARISYIL,Sipariş Grup Adı,Kilo,Net Sevk Kilo,Planlanan Kilo,Metre,Net Sevk Metre,Reçete Adı,İşlem Grubu,İşlem Tipi,Planlanan İmalat Tarihi,Termin Tarihi,Termin Gecikme Gün,İmalat Gecikme Gün,Onay Gün,Reçete Gün,işleme alınma-onay gün,işlem süresi,sevk-işlem,brm boya (kg/m),metre başına düşen boya maliyeti,metre başına düşen boya süresi,Fire Oranı,Reçete Tarihi,Ham Stok Giriş Tarihi,Onay Tarihi,MAKINAID,Fire Oranı.1,Lab Reçete Boya Maliyeti,Boya Maliyeti (Kg),Kimyasal Maliyeti (Kg),En İstenen,Gramaj İstenen,Kazana İlk Giriş Tarihi,Son Çıkış Tarihi,Siparis İşlem Grubu,Sevk İrsaliye Tarihleri
0,2023,2023-01-16,2023,LCW,20.0000,15.2000,20.0000,0,58.9150,R2E 18-3919 TCX.QTX,Renkli,Normal,2023-01-16,2023-01-16,8.0000,4.3603,29,3.7048,4.3603,3.7386,0.0989,0.3395,0.1027,0.0635,0.2400,2023-01-19 16:54:58,2022-12-18,2023-01-16,0050 HT/3,24.0000,9.9035,6.0489,2.4044,185,145,2023-01-20 08:38:53,2023-01-24 02:22:25,Renkli,24.01.2023
1,2023,2023-01-16,2023,LCW,17.0000,14.3000,17.0000,0,44.6880,R2E 18-3919 TCX.QTX,Renkli,Normal,2023-01-16,2023-01-16,8.0000,4.3604,66,3.7048,4.3604,3.7386,0.0990,0.3804,0.1354,0.0837,0.1588,2023-01-19 16:54:58,2022-11-11,2023-01-16,0050 HT/3,15.8800,9.9035,6.0489,2.4044,185,190,2023-01-20 08:38:56,2023-01-24 02:22:35,Renkli,24.01.2023
2,2023,2023-01-16,2023,LCW,10.0000,9.0000,10.0000,0,32.1430,R2E 18-3919 TCX.QTX,Renkli,Normal,2023-01-16,2023-01-16,8.0000,4.3604,102,3.7048,4.3604,3.7387,0.0991,0.3111,0.1882,0.1163,0.1000,2023-01-19 16:54:58,2022-10-06,2023-01-16,0050 HT/3,10.0000,9.9035,6.0489,2.4044,110,260,2023-01-20 08:38:59,2023-01-24 02:22:42,Renkli,24.01.2023
3,2023,2023-01-18,2023,VERY MAIN,306.8000,273.5000,306.8000,0,1005.5150,NAVY BLAZER,Renkli,Normal,2023-01-18,2023-01-11,14.0000,3.8736,-1,3.5086,3.8736,3.1628,0.0364,0.3051,0.0044,0.0031,0.1085,2023-01-21 12:12:24,2023-01-19,2023-01-18,0450 HT/1,10.8500,7.7612,4.4498,3.3353,180,160,2023-01-21 20:58:00,2023-01-25 00:52:26,Renkli,25.01.2023
4,2023,2023-01-18,2023,VERY MAIN,28.0000,25.0000,28.0000,0,68.4930,NAVY BLAZER,Renkli,Normal,2023-01-18,2023-01-11,14.0000,3.8736,1,3.5086,3.8736,3.1629,0.0365,0.4088,0.0650,0.0462,0.1071,2023-01-21 12:12:24,2023-01-17,2023-01-18,0450 HT/1,10.7100,7.7612,4.4498,3.3353,160,220,2023-01-21 20:58:00,2023-01-25 00:52:33,Renkli,25.01.2023


In [167]:
df.dtypes

,0
PARTIYIL,int64
Parti Açılış Tarihi,datetime64[ns]
BOYASIPARISYIL,int64
Sipariş Grup Adı,object
Kilo,float64
Net Sevk Kilo,float64
Planlanan Kilo,float64
Metre,int64
Net Sevk Metre,float64
Reçete Adı,object


In [168]:
# prompt: Sevk İrsaliye Tarihleri, BOYASIPARISYIL, PARTIYIL datatime çevir

import pandas as pd
# Sevk İrsaliye Tarihleri sütununu datetime'a çevirme
df['Sevk İrsaliye Tarihleri'] = pd.to_datetime(df['Sevk İrsaliye Tarihleri'], errors='coerce')

# BOYASIPARISYIL sütununu datetime'a çevirme
df['BOYASIPARISYIL'] = pd.to_datetime(df['BOYASIPARISYIL'], errors='coerce')

# PARTIYIL sütununu datetime'a çevirme
df['PARTIYIL'] = pd.to_datetime(df['PARTIYIL'], errors='coerce')


<ipython-input-168-49e8afa8b01b>:5: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Sevk İrsaliye Tarihleri'] = pd.to_datetime(df['Sevk İrsaliye Tarihleri'], errors='coerce')


In [169]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3120 entries, 0 to 5074
Data columns (total 39 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   PARTIYIL                          3120 non-null   datetime64[ns]
 1   Parti Açılış Tarihi               3120 non-null   datetime64[ns]
 2   BOYASIPARISYIL                    3120 non-null   datetime64[ns]
 3   Sipariş Grup Adı                  3120 non-null   object        
 4   Kilo                              3120 non-null   float64       
 5   Net Sevk Kilo                     3120 non-null   float64       
 6   Planlanan Kilo                    3120 non-null   float64       
 7   Metre                             3120 non-null   int64         
 8   Net Sevk Metre                    3120 non-null   float64       
 9   Reçete Adı                        3120 non-null   object        
 10  İşlem Grubu                       3120 non-null   obj

In [170]:
# prompt: En İstenen, Gramaj İstenen değişkenlerini df den çıkaralım

df = df.drop(['En İstenen', 'Gramaj İstenen'], axis=1)


In [171]:
# prompt: yeni değişkenler ekleyelim

# Sipariş ile teslimat arasındaki farkı hesaplama
df['Teslimat_Suresi'] = (df['Son Çıkış Tarihi'] - df['BOYASIPARISYIL']).dt.days

# Parti açılış tarihi ile sevk irsaliye tarihi arasındaki farkı hesaplama
df['Parti_Sevk_Farki'] = (df['Sevk İrsaliye Tarihleri'] - df['Parti Açılış Tarihi']).dt.days

# Planlanan imalat tarihi ile termin tarihi arasındaki farkı hesaplama
df['Planlanan_Termin_Farki'] = (df['Termin Tarihi'] - df['Planlanan İmalat Tarihi']).dt.days

# Reçete tarihi ile kazana ilk giriş tarihi arasındaki farkı hesaplama
df['Reçete_Kazan_Farki'] = (df['Kazana İlk Giriş Tarihi'] - df['Reçete Tarihi']).dt.days

# Ham stok giriş tarihi ile onay tarihi arasındaki farkı hesaplama
df['Ham_Stok_Onay_Farki'] = (df['Onay Tarihi'] - df['Ham Stok Giriş Tarihi']).dt.days



In [172]:
df.isnull().sum()

,0
PARTIYIL,0
Parti Açılış Tarihi,0
BOYASIPARISYIL,0
Sipariş Grup Adı,0
Kilo,0
Net Sevk Kilo,0
Planlanan Kilo,0
Metre,0
Net Sevk Metre,0
Reçete Adı,0


In [173]:
# prompt: label encoding yapalım

from sklearn.preprocessing import LabelEncoder

# LabelEncoder nesnesi oluşturma
le = LabelEncoder()

# Kategorik sütunları seçme
categorical_cols = df.select_dtypes(include=['object']).columns

# Kategorik sütunlara Label Encoding uygulama
for col in categorical_cols:
  df[col] = le.fit_transform(df[col])

df.head()


,PARTIYIL,Parti Açılış Tarihi,BOYASIPARISYIL,Sipariş Grup Adı,Kilo,Net Sevk Kilo,Planlanan Kilo,Metre,Net Sevk Metre,Reçete Adı,İşlem Grubu,İşlem Tipi,Planlanan İmalat Tarihi,Termin Tarihi,Termin Gecikme Gün,İmalat Gecikme Gün,Onay Gün,Reçete Gün,işleme alınma-onay gün,işlem süresi,sevk-işlem,brm boya (kg/m),metre başına düşen boya maliyeti,metre başına düşen boya süresi,Fire Oranı,Reçete Tarihi,Ham Stok Giriş Tarihi,Onay Tarihi,MAKINAID,Fire Oranı.1,Lab Reçete Boya Maliyeti,Boya Maliyeti (Kg),Kimyasal Maliyeti (Kg),Kazana İlk Giriş Tarihi,Son Çıkış Tarihi,Siparis İşlem Grubu,Sevk İrsaliye Tarihleri,Teslimat_Suresi,Parti_Sevk_Farki,Planlanan_Termin_Farki,Reçete_Kazan_Farki,Ham_Stok_Onay_Farki
0,1970-01-01 00:00:00.000002023,2023-01-16,1970-01-01 00:00:00.000002023,39,20.0000,15.2000,20.0000,0,58.9150,426,3,0,2023-01-16,2023-01-16,8.0000,4.3603,29,3.7048,4.3603,3.7386,0.0989,0.3395,0.1027,0.0635,0.2400,2023-01-19 16:54:58,2022-12-18,2023-01-16,3,24.0000,9.9035,6.0489,2.4044,2023-01-20 08:38:53,2023-01-24 02:22:25,3,2023-01-24,19381,8,0,0,29
1,1970-01-01 00:00:00.000002023,2023-01-16,1970-01-01 00:00:00.000002023,39,17.0000,14.3000,17.0000,0,44.6880,426,3,0,2023-01-16,2023-01-16,8.0000,4.3604,66,3.7048,4.3604,3.7386,0.0990,0.3804,0.1354,0.0837,0.1588,2023-01-19 16:54:58,2022-11-11,2023-01-16,3,15.8800,9.9035,6.0489,2.4044,2023-01-20 08:38:56,2023-01-24 02:22:35,3,2023-01-24,19381,8,0,0,66
2,1970-01-01 00:00:00.000002023,2023-01-16,1970-01-01 00:00:00.000002023,39,10.0000,9.0000,10.0000,0,32.1430,426,3,0,2023-01-16,2023-01-16,8.0000,4.3604,102,3.7048,4.3604,3.7387,0.0991,0.3111,0.1882,0.1163,0.1000,2023-01-19 16:54:58,2022-10-06,2023-01-16,3,10.0000,9.9035,6.0489,2.4044,2023-01-20 08:38:59,2023-01-24 02:22:42,3,2023-01-24,19381,8,0,0,102
3,1970-01-01 00:00:00.000002023,2023-01-18,1970-01-01 00:00:00.000002023,72,306.8000,273.5000,306.8000,0,1005.5150,369,3,0,2023-01-18,2023-01-11,14.0000,3.8736,-1,3.5086,3.8736,3.1628,0.0364,0.3051,0.0044,0.0031,0.1085,2023-01-21 12:12:24,2023-01-19,2023-01-18,17,10.8500,7.7612,4.4498,3.3353,2023-01-21 20:58:00,2023-01-25 00:52:26,3,2023-01-25,19382,7,-7,0,-1
4,1970-01-01 00:00:00.000002023,2023-01-18,1970-01-01 00:00:00.000002023,72,28.0000,25.0000,28.0000,0,68.4930,369,3,0,2023-01-18,2023-01-11,14.0000,3.8736,1,3.5086,3.8736,3.1629,0.0365,0.4088,0.0650,0.0462,0.1071,2023-01-21 12:12:24,2023-01-17,2023-01-18,17,10.7100,7.7612,4.4498,3.3353,2023-01-21 20:58:00,2023-01-25 00:52:33,3,2023-01-25,19382,7,-7,0,1


In [174]:
# prompt: Standartlaştırma yapalım

# MinMaxScaler nesnesi oluşturma
scaler = MinMaxScaler()

# Sayısal sütunları seçme
numeric_cols = df.select_dtypes(include=['number']).columns

# Sayısal sütunlara MinMaxScaler uygulama
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

df.head()


,PARTIYIL,Parti Açılış Tarihi,BOYASIPARISYIL,Sipariş Grup Adı,Kilo,Net Sevk Kilo,Planlanan Kilo,Metre,Net Sevk Metre,Reçete Adı,İşlem Grubu,İşlem Tipi,Planlanan İmalat Tarihi,Termin Tarihi,Termin Gecikme Gün,İmalat Gecikme Gün,Onay Gün,Reçete Gün,işleme alınma-onay gün,işlem süresi,sevk-işlem,brm boya (kg/m),metre başına düşen boya maliyeti,metre başına düşen boya süresi,Fire Oranı,Reçete Tarihi,Ham Stok Giriş Tarihi,Onay Tarihi,MAKINAID,Fire Oranı.1,Lab Reçete Boya Maliyeti,Boya Maliyeti (Kg),Kimyasal Maliyeti (Kg),Kazana İlk Giriş Tarihi,Son Çıkış Tarihi,Siparis İşlem Grubu,Sevk İrsaliye Tarihleri,Teslimat_Suresi,Parti_Sevk_Farki,Planlanan_Termin_Farki,Reçete_Kazan_Farki,Ham_Stok_Onay_Farki
0,1970-01-01 00:00:00.000002023,2023-01-16,1970-01-01 00:00:00.000002023,0.5132,0.0093,0.0079,0.0097,0.0000,0.0001,0.8053,0.5000,0.0000,2023-01-16,2023-01-16,0.4844,0.0371,0.1098,0.0261,0.0371,0.0127,0.0007,0.0163,0.0001,0.0001,0.2408,2023-01-19 16:54:58,2022-12-18,2023-01-16,0.0968,0.2408,0.2679,0.0278,0.0007,2023-01-20 08:38:53,2023-01-24 02:22:25,0.6000,2023-01-24,0.0479,0.0884,0.5029,0.7329,0.1098
1,1970-01-01 00:00:00.000002023,2023-01-16,1970-01-01 00:00:00.000002023,0.5132,0.0078,0.0074,0.0083,0.0000,0.0001,0.8053,0.5000,0.0000,2023-01-16,2023-01-16,0.4844,0.0371,0.1824,0.0261,0.0371,0.0127,0.0007,0.0182,0.0001,0.0001,0.1594,2023-01-19 16:54:58,2022-11-11,2023-01-16,0.0968,0.1594,0.2679,0.0278,0.0007,2023-01-20 08:38:56,2023-01-24 02:22:35,0.6000,2023-01-24,0.0479,0.0884,0.5029,0.7329,0.1824
2,1970-01-01 00:00:00.000002023,2023-01-16,1970-01-01 00:00:00.000002023,0.5132,0.0044,0.0047,0.0049,0.0000,0.0000,0.8053,0.5000,0.0000,2023-01-16,2023-01-16,0.4844,0.0371,0.2529,0.0261,0.0371,0.0127,0.0007,0.0149,0.0001,0.0002,0.1004,2023-01-19 16:54:58,2022-10-06,2023-01-16,0.0968,0.1004,0.2679,0.0278,0.0007,2023-01-20 08:38:59,2023-01-24 02:22:42,0.6000,2023-01-24,0.0479,0.0884,0.5029,0.7329,0.2529
3,1970-01-01 00:00:00.000002023,2023-01-18,1970-01-01 00:00:00.000002023,0.9474,0.1500,0.1417,0.1491,0.0000,0.0015,0.6975,0.5000,0.0000,2023-01-18,2023-01-11,0.4929,0.0322,0.0510,0.0245,0.0322,0.0107,0.0003,0.0146,0.0000,0.0000,0.1089,2023-01-21 12:12:24,2023-01-19,2023-01-18,0.5484,0.1089,0.2100,0.0205,0.0009,2023-01-21 20:58:00,2023-01-25 00:52:26,0.6000,2023-01-25,0.0505,0.0854,0.4928,0.7329,0.0510
4,1970-01-01 00:00:00.000002023,2023-01-18,1970-01-01 00:00:00.000002023,0.9474,0.0132,0.0129,0.0136,0.0000,0.0001,0.6975,0.5000,0.0000,2023-01-18,2023-01-11,0.4929,0.0322,0.0549,0.0245,0.0322,0.0107,0.0003,0.0196,0.0000,0.0001,0.1075,2023-01-21 12:12:24,2023-01-17,2023-01-18,0.5484,0.1075,0.2100,0.0205,0.0009,2023-01-21 20:58:00,2023-01-25 00:52:33,0.6000,2023-01-25,0.0505,0.0854,0.4928,0.7329,0.0549


In [175]:
# prompt: y termin tarihi x kalan değişkenler olarak atayalım

# 'Termin Tarihi' sütununu hedef değişken olarak ayarla
y = df["Sevk İrsaliye Tarihleri"]

# Diğer sütunları bağımsız değişkenler olarak ayarla
X = df.drop('Sevk İrsaliye Tarihleri', axis=1)


In [176]:
# X ve y boyutlarını kontrol edin
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

# İlk birkaç satırı kontrol edelim
print(X.head())
print(y.head())


X shape: (3120, 41)
y shape: (3120,)
                       PARTIYIL Parti Açılış Tarihi                BOYASIPARISYIL  Sipariş Grup Adı   Kilo  Net Sevk Kilo  Planlanan Kilo  Metre  Net Sevk Metre  Reçete Adı  İşlem Grubu  İşlem Tipi Planlanan İmalat Tarihi Termin Tarihi  Termin Gecikme Gün  İmalat Gecikme Gün  Onay Gün  Reçete Gün  işleme alınma-onay gün  işlem süresi  sevk-işlem   brm boya (kg/m)  metre başına düşen boya maliyeti  metre başına düşen boya süresi  Fire Oranı       Reçete Tarihi Ham Stok Giriş Tarihi Onay Tarihi  \
0 1970-01-01 00:00:00.000002023          2023-01-16 1970-01-01 00:00:00.000002023            0.5132 0.0093         0.0079          0.0097 0.0000          0.0001      0.8053       0.5000      0.0000              2023-01-16    2023-01-16              0.4844              0.0371    0.1098      0.0261                  0.0371        0.0127       0.0007           0.0163                            0.0001                          0.0001      0.2408 2023-01-19 16:54:5

In [177]:
# prompt:  Tarih sütunlarını gün sayısına dönüştürme

# Tarih sütunlarını seçme
date_cols = ['Parti Açılış Tarihi', 'Planlanan İmalat Tarihi', 'Termin Tarihi', 'Reçete Tarihi',
              'Ham Stok Giriş Tarihi', 'Onay Tarihi', 'Kazana İlk Giriş Tarihi', 'Son Çıkış Tarihi',
              'Sevk İrsaliye Tarihleri', 'BOYASIPARISYIL', 'PARTIYIL']

# Referans tarihi belirleme (örneğin, en eski tarih)
reference_date = df['Parti Açılış Tarihi'].min()

# Tarih sütunlarını gün sayısına dönüştürme
for col in date_cols:
  df[col] = (df[col] - reference_date).dt.days


In [178]:
# Özellikler ve hedef değişkeni belirleme
X = df.drop(columns=["Sevk İrsaliye Tarihleri"])  # 'target' yerine tahmin etmek istediğiniz hedef değişkeni koyun
y = df['Sevk İrsaliye Tarihleri']

# Eğitim ve test setlerine ayırma
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Linear Regression modelini oluşturma ve eğitim
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

model = LinearRegression()
model.fit(X_train, y_train)

# Test verileri üzerinde tahmin yapma
y_pred = model.predict(X_test)

# Modelin performansını değerlendirme
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2): {r2}")


Mean Squared Error (MSE): 1.3465362108376841e-25
R-squared (R2): 1.0


In [184]:
# prompt: gradient boosting deneyelim

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Gradient Boosting modelini oluşturma ve eğitim
gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
gb_model.fit(X_train, y_train)

# Test verileri üzerinde tahmin yapma
y_pred_gb = gb_model.predict(X_test)

# Modelin performansını değerlendirme
mse_gb = mean_squared_error(y_test, y_pred_gb)
r2_gb = r2_score(y_test, y_pred_gb)


In [192]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from datetime import datetime, timedelta
import numpy as np

# Varsayılan model (gerçek modeliniz yerine geçer)
# gb_model = ...  # Modelinizi burada yüklemelisiniz, örneğin pickle ile

# Kullanıcıdan input alma
try:
    # Kullanıcıdan veri al
    kilo = float(input("Kilo değerini girin: "))
    metre = float(input("Metre değerini girin: "))
    islem_grubu = input("İşlem grubu değerini girin (Örn: Renkli, Boyasız): ")

    # Input değerlerini kullanarak yeni bir dataframe oluşturma
    new_data = pd.DataFrame({
        'Kilo': [kilo],
        'Metre': [metre],
        'İşlem Grubu': [islem_grubu]
    })

    # Kategorik sütunları seçme
    categorical_cols = ['İşlem Grubu']

    # LabelEncoder nesnesi oluşturma
    le = LabelEncoder()

    # Kategorik sütunlara Label Encoding uygulama
    new_data['İşlem Grubu'] = le.fit_transform(new_data['İşlem Grubu'])

    # Sayısal sütunları seçme
    numeric_cols = ['Kilo', 'Metre']

    # MinMaxScaler nesnesi oluşturma
    scaler = MinMaxScaler()

    # Sayısal sütunlara MinMaxScaler uygulama
    new_data[numeric_cols] = scaler.fit_transform(new_data[numeric_cols])

    # Tahmin yapma (Modelinize uygun satırı açmalısınız)
    # prediction = gb_model.predict(new_data)

    # Örnek tahmin değeri (Modeliniz olmadığı için tahmini simüle ediyorum)
    prediction = np.array([3])  # Örnek tahmin sonucu: 3 gün

    # Bugünün tarihi
    current_date = datetime.now()

    # Tahmin edilen gün sayısına göre gelecekteki tarihi hesaplama
    future_date = current_date + timedelta(days=int(prediction[0]))

    # Tarihi formatlayarak yazdırma (GG.AA.YYYY formatında)
    print(f"Sevk İrsaliye Tarihi tahmini: {future_date.strftime('%d.%m.%Y')}")

except ValueError:
    print("Lütfen geçerli bir sayı girin.")
except Exception as e:
    print(f"Bir hata oluştu: {e}")



Kilo değerini girin: 120
Metre değerini girin: 120
İşlem grubu değerini girin (Örn: Renkli, Boyasız): Renkli
Sevk İrsaliye Tarihi tahmini: 09.09.2024
